In [1]:
import os
import torch
from torch import nn
from d2l import torch as d2l

# 词的相似性和类比任务
之前在一个小型的数据集上训练一个`word2vec`模型, 并且使用这一个模型作为输入词寻找语义相似的词, 实际上, 在大型语料库中预先训练的词向量可以应用于下游的 `NLP` 任务, 这里展示如何使用大型语料库中训练词向量的语义, 并且把预训练的词向量应用到词的相似性和类比任务上

## 加载预训练词向量
如下都是维度为 `50、100、300`的预训练`Glove`嵌入, 预训练的`fastText`由多种语言

In [2]:
#@save
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')

#@save
d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')

#@save
d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')

#@save
d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')

In [4]:
# 用于加载预训练的 GloVe 以及 fastText
#@save
class TokenEmbedding:
    """GloVe嵌入"""
    def __init__(self, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(
            embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {token: idx for idx, token in
                             enumerate(self.idx_to_token)}

    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe网站：https://nlp.stanford.edu/projects/glove/
        # fastText网站：https://fasttext.cc/
        with open(os.path.join(data_dir, 'vec.txt'), 'r') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # 跳过标题信息，例如fastText中的首行
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [self.token_to_idx.get(token, self.unknown_idx)
                   for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

In [5]:
# 加载 50 维 GloVe 嵌入
glove_6b50d = TokenEmbedding('glove.6b.50d')

In [6]:
len(glove_6b50d)

400001

In [7]:
# 获取一个单词的索引
glove_6b50d.token_to_idx['beautiful'], glove_6b50d.idx_to_token[3367]

(3367, 'beautiful')

## 引用预训练词向量

### 词相似度
使用 `knn` 模型来估计词相似度

In [8]:
def knn(W, x, k):
    # 增加1e-9以获得数值稳定性
    cos = torch.mv(W, x.reshape(-1,)) / (
        torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) *
        torch.sqrt((x * x).sum()))
    _, topk = torch.topk(cos, k=k)
    return topk, [cos[int(i)] for i in topk]

In [9]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):  # 排除输入词
        print(f'{embed.idx_to_token[int(i)]}：cosine相似度={float(c):.3f}')

In [10]:
get_similar_tokens('chip', 3, glove_6b50d)

chips：cosine相似度=0.856
intel：cosine相似度=0.749
electronics：cosine相似度=0.749


In [11]:
get_similar_tokens('beautiful', 3, glove_6b50d)

lovely：cosine相似度=0.921
gorgeous：cosine相似度=0.893
wonderful：cosine相似度=0.830


### 词类比

In [12]:
# 分析词类比
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed[[token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[int(topk[0])]  # 删除未知词

In [13]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

In [15]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'japan'

## 总结
之前提到的 `word2vec` 或者 `GloVe` 模型可能实现起来比较困难, 但是对于这些模型, 如果我们仅仅在一个比较小的语义库上进行训练, 可能得到的嵌入模型并不是最好的, 所以在之后的任务中, 最好利用预训练的词嵌入模型来应用于下游的 `NLP` 任务